## Нахождение мест посадки пассажиров по транзакционным данным об облате проезда смарт-картами "Подорожник"                            

В основе данной методики лежит метод восстановления информации об остановках, где садились пассажиры, по транзакциям оплаты проезда смарт-картами. Особенность имеющихся транзакций - отсутствие информации о месте их совершения.

Базовое предположение: транзакции оплаты совершаются сразу при входе пассажира в транспортное средство.

Созданный алгоритм базируется на идеях, описанных в статье «Extracting bus transit boarding stop information using smart card transaction data» (Chen Z., Fan W.; 2018) и адаптирован под особенности системы коммерческого транспорта Санкт-Петербурга.

In [1]:
import numpy as np
import pandas as pd

## Обработка и анализ данных. Проверка корректности

In [2]:
# Загружаем dataset с информацией об исследуемых коммерческих маршрутах
comm_df = pd.read_csv('commerce_stops.csv')
comm_df = comm_df[['route_number', 'num1', 'stops_in_direction1', 'num2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2', 'distances1', 'distanсes2', 'sсheduled_trip_time']]

In [3]:
# Вспомогательная функция, вычисляющая время в пути указанного маршрута между всеми парами остановок
# Результатом ее работы становится обновление информации в dataset-е коммерческих маршрутов 'comm_df'
def get_time_between_stations(route_num):
    stations_in_dist1 = np.array(comm_df[comm_df['route_number'] == route_num]['distances1'].values[0].split(',')).astype(int)
    stations_in_dist2 = np.array(comm_df[comm_df['route_number'] == route_num]['distanсes2'].values[0].split(',')).astype(int)
    
    all_route_dist1 = np.sum(stations_in_dist1)
    all_route_dist2 = np.sum(stations_in_dist2)
    
    all_route_time = comm_df[comm_df['route_number'] == route_num]['sсheduled_trip_time'].values[0]
    
    avg_speed1 = all_route_dist1 * 1.0 / all_route_time # в м/с
    avg_speed2 = all_route_dist1 * 1.0 / all_route_time # в м/с
    
    times_array1 = str(int(round((stations_in_dist1[0] * 1.0) /  avg_speed1)))
    times_array2 = str(int(round((stations_in_dist2[0] * 1.0) /  avg_speed2)))
    
    for i in range(1, stations_in_dist1.shape[0]):
        times_array1 += ',' + str(int(round((stations_in_dist1[i] * 1.0) /  avg_speed1)))
    for i in range(1, stations_in_dist2.shape[0]):
        times_array2 += ',' + str(int(round((stations_in_dist2[i] * 1.0) /  avg_speed2)))
        
    return times_array1, times_array2

times_for_stops1_arr = []
times_for_stops2_arr = []

all_comm_routes = comm_df['route_number'].values
for route in all_comm_routes:
    t1, t2 = get_time_between_stations(route)
    times_for_stops1_arr.append(t1)
    times_for_stops2_arr.append(t2)
    

# Добавление информации о временах преодоления остановок в dataframe
times_for_stops1_column = pd.Series(np.array(times_for_stops1_arr), index=comm_df.index)
comm_df.loc[:, 'times_for_stops1'] = times_for_stops1_column

times_for_stops2_column = pd.Series(np.array(times_for_stops2_arr), index=comm_df.index)
comm_df.loc[:, 'times_for_stops2'] = times_for_stops2_column

# Сохраняем обновленную информацию о коммерческих маршрутах
comm_df.to_csv('commercial_routes.csv', encoding='utf-8-sig')

In [4]:
# Перезагружаем обновленный dataset с коммерческими маршрутами для дальнейшего использования
comm_df = pd.read_csv('commercial_routes.csv')
comm_df = comm_df[['route_number', 'num1', 'stops_in_direction1', 'num2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2', 'distances1', 'distanсes2', 'sсheduled_trip_time', 'times_for_stops1', 'times_for_stops2']]
comm_df

,route_number,num1,stops_in_direction1,num2,stops_in_direction2,clusters_in_direction1,clusters_in_direction2,distances1,distanсes2,sсheduled_trip_time,times_for_stops1,times_for_stops2
0,191,38,"20777,3391,3471,3711,3713,7070,4603,3717,2675,...",34,"21667,3463,14949,1351,1531,2186,2223,1788,4376...","2058,2210,2338,1940,1752,2529,1712,1716,603,60...","416,2325,1364,294,860,2179,1020,1498,1018,876,...","375,323,400,575,691,398,750,309,331,446,431,50...","557,175,370,522,401,644,448,371,480,423,683,30...",3960,"89,77,95,137,164,95,178,73,79,106,102,121,101,...","132,42,88,124,95,153,106,88,114,100,162,72,78,..."
1,68,19,"20044,2263,15310,3416,2350,15321,2197,20058,25...",19,"20027,20028,20030,20035,20037,20040,1830,3540,...","443,2287,2096,2249,1674,2506,2197,1672,214,167...","2462,1879,2366,1407,1673,2296,1675,1670,2219,1...","957,409,629,312,283,471,453,779,428,550,512,54...","383,306,346,391,357,386,546,502,606,332,725,50...",1920,"218,93,144,71,65,108,103,178,98,126,117,125,83...","87,70,79,89,82,88,125,115,138,76,166,116,109,7..."
2,1,14,"24748,2909,3303,16431,1632,16501,1792,22153,45...",12,"3201,2778,2127,3716,4517,22153,1792,22353,1632...","1227,1319,2048,1080,1081,1305,1309,249,268,112...","1270,942,1196,1125,268,249,1309,1081,1081,2048...","380,263,282,254,365,269,581,248,277,205,388,47...","320,1570,394,360,345,571,458,245,460,230,422",720,"55,38,41,37,53,39,84,36,40,30,56,68,145","46,226,57,52,50,82,66,35,66,33,61"
3,184,17,"20044,2263,15310,3416,2350,15321,2197,20058,25...",17,"28233,23019,14743,3489,1830,3540,2211,2293,254...","443,2287,2096,2249,1674,2506,2197,1672,214,167...","2297,2463,2367,2368,1675,1670,2219,1671,214,16...","944,414,634,317,284,467,451,776,425,558,512,55...","230,839,313,367,559,512,558,425,776,451,467,28...",1800,"210,92,141,71,63,104,100,173,95,124,114,124,82...","51,187,70,82,124,114,124,95,173,100,104,63,71,..."
4,226,36,"22739,22739,1542,2893,3298,16510,2583,2086,238...",35,"27893,27888,27885,22084,22085,22086,22087,2222...","698,698,241,1241,370,369,1230,1269,2194,219,21...","2142,1677,2243,1676,1678,1109,1112,1110,1111,1...","394,352,599,512,457,592,503,390,366,144,554,31...","385,426,229,237,471,448,505,496,365,511,379,45...",2880,"74,66,112,96,86,111,94,73,69,27,104,60,96,114,...","72,80,43,45,88,84,95,93,69,96,71,85,64,47,59,9..."
5,92,13,"15952,1847,4116,3485,1556,3116,1921,3114,3749,...",11,"22798,2475,3421,3114,1921,3116,1556,3485,4116,...","193,1698,2361,1768,914,1770,1738,1769,929,929,...","2453,2257,929,1769,1738,1770,914,1768,2361,169...","1940,363,465,672,326,408,655,373,233,692,465,211","749,723,613,680,322,412,470,431,562,1920",1860,"530,99,127,184,89,112,179,102,64,189,127,58","205,198,168,186,88,113,129,118,154,525"
6,55,7,"4502,1835,1591,4537,1360,2641,1548",7,"1548,24981,1360,4499,2798,3022,4502","378,1001,1000,1957,329,469,903","903,469,329,1957,1000,1001,378","697,700,321,385,802,900","893,827,338,429,656,585",840,"154,155,71,85,177,199","197,183,75,95,145,129"
7,67,12,"1361,3501,3577,2904,3302,2866,3163,2685,2684,3...",12,"18397,25347,1473,1474,1971,1662,1707,4769,1706...","332,2380,2457,1300,2047,1156,1862,633,188,761,...","954,761,188,633,1862,1156,1301,2047,1300,2457,...","514,265,386,255,794,494,353,342,729,1180,857","1950,1360,729,342,353,494,794,255,386,265,514",1380,"115,59,86,57,178,111,79,77,163,264,192","436,304,163,77,79,111,178,57,86,59,115"
8,7,8,"3201,2957,2652,1722,2858,2984,16362,18467",6,"18467,16362,2984,2858,1722,2957,3201","1270,494,395,1330,1126,1514,1313,1309","1313,1514,1126,1330,494,1270","454,154,341,456,368,386,272","411,390,551,409,619",360,"67,23,50,68,54,57,40","61,58,82,61,92"


In [5]:
# Загружаем обработанный dataset с транзакциями
tr_df = pd.read_csv('final_transactions_with_duplicate_labels_data.csv')

In [6]:
# Удаляем дубликаты из данных
# Анализ данных на дубликаты проводился в 'data_preparation.py'
tr_df = tr_df[tr_df['IS_DUPLICATE'] == 0]

In [7]:
# Проводим анализ записей и выводим некоторую статистику по данным

# Отсортированные по убыванию количества транзакций записи о маршруте, номере бортового транспорта и днях оплаты
max_transactions = tr_df.groupby(['ROUTE_NUM', 'CARRIER_BOARD_NUM', 'TRANSACT_MONTH', 'TRANSACT_DAY']).TRANSACT_TIME.count()
max_transactions = max_transactions.sort_values(ascending=False)

### NB! Некорректные значения в данных о транзакциях!
В результате анализа данных транзакций были обнаружены некорректные записи.
На коммерческом маршруте №68 количество дневных транзакций оплаты по картам у транспортного средства с бортовым номером 0 аномально большое. Потенциальной причиной возникновения некорректных данных можно считать сбой в системе в октябре 2018 года, который привел к потере информации о бортовых номерах у ряда транспортных средств, следующих по маршруту №68 и в результате произошло объединение данных о нескольких транспортных средствах (с новым общим бортовым номером 0).

Т.к. разделить такие данные не представляется возможным, их нельзя учитывать при расчете пассажиропотока по этой методике.

In [8]:
# Готовим датасеты для анализа выбранных маршрутов
choosen_routes = ['68', '191', '92', '7', '1', '55', '67', '184', '226']

for route in choosen_routes:
    route_df = tr_df[tr_df['ROUTE_NUM'] == route]
    route_df.to_csv('transactions_by_routes/' + route + '_route.csv')

## Алгоритм вычисления мест посадки пассажиров

Данный алгоритм состоит из нескольких этапов. 

1. На первом этапе происходит разделение поездок. Каждой отдельной поездке назначается свой идентификатор. Отделение различных поездок друг от друга происходит по следующему принципу (см. блок-схему). Здесь параметр 12 минут был выбран практически на основе анализа имеющихся данных как оптимальный разделитель.

 ![choose_trip.png](choose_trip.png)
 
2. На втором этапе производится кластеризация записей. Маленький временной промежуток между транзакциями означает, что совершившие их пассажиры садились в одном месте, а значит такие записи нужно отнести к одному кластеру. Более подробно данный этап описан на следующей блок-схеме:

 ![clasterization_diagram.png](clasterization_diagram.png)

3. Целью третьего этапа является соотнесение полученных кластеров и остановок на маршруте. В результате для каждой транзакции должна появиться информация об остановке, на которой входил совершивший ее пассажир.
 Алгоритм соотнесения кластеров и остановок выглядит следующим  образом:
 1. Первому кластеру каждой поездки ставится в соответствие первая остановка маршрута (это базовое предположение, его можно попробовать поменять, если станет известна дополнительная информация о том, как это можно вычислить более точно).
 2. Обозначим за $t_{n(n+1)}$ временную разницу между первой записью в $n+1$-ом кластере и последней записью в $n$-ом кластере. Пусть кластер $n$ был соотнесен с остановкой $i$. Обозначим за $a_{i(i+1)}$ время прохождения транспортным средством по маршруту от остановки $i$ до остановки $i+1$.
 3. Если $t_{n(n+1)}<a_{i(i+1)}$, то соотнесем кластер $n+1$ с остановкой $i+1$.
 4. Если $t_{n(n+1)}>a_{i(i+1)}$, то сравним $t_{n(n+1)}$ и $a_{i(i+2)}$. Если снова больше, то будем сравнивать $t_{n(n+1)}$ с $a_{i(i+k)}$ до тех пор, пока не будет выполнено $t_{n(n+1)}<a_{i(i+k)}$. После этого соотнесем кластер $n+1$ с остановкой $i+k$.
 
 Пример для наглядности:
 
 ![boarding_extraction.png](boarding_extraction.png)
 
Результатом работы описанного выше алгоритма будет нахождение для каждой транзакционной записи номера и направления поездки, а также остановки, на которой садился пассажир, ее совершивший.

In [9]:
# Здесь задаются параметры алгоритму вычисления мест посадки пассажиров

# 1. Параметр, предназначенный для грубого разделения поездок транспортного средства по маршруту
# его значение - временной промежуток в секундах между записями об оплате, 
# который позволяет гарантированно отнести эти записи к разным поездкам
coarse_trips_pivot = 720 

# 2. Параметр, позволяющий понять, принадлежат ли последовательные записи одному кластеру или нет
# его значение также в секундах
clusters_pivot = 25

In [10]:
# Вспомогательная функция, вычисляющая разность в секундах между двумя переданными временными штампами
# Допущения: timestamp2 > timestamp1, это гарантируется при подаче на вход
def get_times_delta(timestamp1, timestamp2):
    time_arr1 = timestamp1.split(':')
    time_arr2 = timestamp2.split(':')
    return (int(time_arr2[2]) + int(time_arr2[1]) * 60 + int(time_arr2[0]) * 3600) - (int(time_arr1[2]) + int(time_arr1[1]) * 60 + int(time_arr1[0]) * 3600)

#get_times_delta("05:23:00", "05:24:00")

In [11]:
# Вспомогательная функция. Возвращает dataframe с записями об указанном маршруте в указанную дату
# В данной функции дополнительно происходит проверка на корректность информации о номере т\с
# Корректные номера т\с являются однозначными - четырехзначными. Номера, содержащие большее количество цифр - некорректны

# Вспомогательная функция. Проверяет, есть ли в указанную дату записи о данном маршруте
# Вспомогательная функция. Позволяет по номеру маршрута узнать длительность его поездки по маршруту

def get_route_df_by_date(route_num, tr_date):
    route_df = pd.read_csv('transactions_by_routes/' + route_num + '_route.csv')
    route_df = route_df[route_df['TRANSACT_MONTH'] == int(tr_date.split('.')[1])]
    route_df = route_df[route_df['TRANSACT_DAY'] == int(tr_date.split('.')[0])] 
    route_df = route_df[route_df['CARRIER_BOARD_NUM'] <= 10000]
    route_df = route_df.sort_values(by=['TRANSACT_YEAR', 'TRANSACT_MONTH', 'TRANSACT_DAY', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME'])
    route_df = route_df.reset_index(drop=True)
    return route_df[['CARD_NUM', 'TRANSACT_DAY', 'TRANSACT_MONTH', 'TRANSACT_YEAR', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME', 
                     'ROUTE_NUM']]


def check_date(route_num, tr_date):
    route_df = get_route_df_by_date(route_num, tr_date)
    return route_df['ROUTE_NUM'].count() > 0


def get_scheduled_trip_time(route_num):
    return comm_df[comm_df['route_number'] == int(route_num)]['sсheduled_trip_time'].values[0]

In [12]:
# Вспомогательная функция. Позволяет по номеру маршрута узнать длительность его поездки по маршруту
def get_scheduled_trip_time(route_num):
    return comm_df[comm_df['route_number'] == int(route_num)]['sсheduled_trip_time'].values[0]

#get_scheduled_trip_time(68)

In [19]:
# Вспомогательная функция. Для переданного маршрута и дня рассчитывает пассажиропотоки в течение этого дня 
# на всех транспортных средствах, которые были на маршруте в этот день

def get_boarding_locations_by_day(route_num, tr_date):
    cur_comm_df = comm_df[comm_df['route_number'] == int(route_num)]
    route_df = get_route_df_by_date(route_num, tr_date)
    board_nums_array = route_df['CARRIER_BOARD_NUM'].unique()
    for board_num in board_nums_array:
        board_df = route_df[route_df['CARRIER_BOARD_NUM'] == board_num]
        tr_amount = board_df.shape[0]
        tr_times = board_df['TRANSACT_TIME'].values
        
        # Этап 1 - разделение поездок по времени между транзакциями
        board_trips = [1]
        board_directions = [0]
        trip_id = 1
        direction = 0
        first_note_of_the_trip = 0
        for i in range(1, tr_amount):
            sheduled_trip_duration = get_scheduled_trip_time(route_num)
            if (get_times_delta(tr_times[i - 1], tr_times[i]) >= coarse_trips_pivot) or get_times_delta(tr_times[first_note_of_the_trip], tr_times[i]) >= sheduled_trip_duration:
                trip_id += 1
                first_note_of_the_trip = i
                board_trips.append(trip_id)  
                if get_times_delta(tr_times[i - 1], tr_times[i]) >= sheduled_trip_duration:
                    dir_coeff = get_times_delta(tr_times[i - 1], tr_times[i]) // sheduled_trip_duration
                    if dir_coeff % 2 == 0:
                        direction = (direction + 1) % 2
                        board_directions.append(direction)
                    else:
                        board_directions.append(direction)
                else:
                    direction = (direction + 1) % 2
                    board_directions.append(direction)
            else:
                board_trips.append(trip_id)
                board_directions.append(direction)
                
        preliminary_trip_id_column = np.array(board_trips)
        preliminary_direction_column = np.array(board_directions)
        
        # Этап 2 - кластеризация записей
        tr_times = board_df['TRANSACT_TIME'].values
        board_clusters = [1]
        cluster_id = 1
        cur_trip_num = preliminary_trip_id_column[0]
        for i in range(1, tr_amount):
            trip_id = preliminary_trip_id_column[i]
            if trip_id == cur_trip_num:
                if get_times_delta(tr_times[i - 1], tr_times[i]) < clusters_pivot:
                    board_clusters.append(cluster_id)
                else:
                    cluster_id += 1
                    board_clusters.append(cluster_id)
            else:
                cur_trip_num = trip_id
                cluster_id = 1
                board_clusters.append(cluster_id)
        
        clusters_column = np.array(board_clusters)
        
        # Этап 3 - соотнесение кластеров и информации об остановках
        tr_times = board_df['TRANSACT_TIME'].values
        boarding_stations_nums = [0]
        station_num = 0
        cur_trip_num = preliminary_trip_id_column[0]
        cur_direction = preliminary_direction_column[0]
        cur_cluster = clusters_column[0]
        
        if cur_direction == 0:
            stops_times = cur_comm_df['times_for_stops1'].values[0].split(',')
            route_stops_arr = cur_comm_df['stops_in_direction1'].values[0].split(',')
        else:
            stops_times = cur_comm_df['times_for_stops2'].values[0].split(',')
            route_stops_arr = cur_comm_df['stops_in_direction2'].values[0].split(',')
        
        boarding_stations_id = [route_stops_arr[0]]
        station_id = route_stops_arr[0]
              
        for i in range(1, tr_amount):
            trip_id = preliminary_trip_id_column[i]
            direction_id = preliminary_direction_column[i]
            cluster_id = clusters_column[i]
            if trip_id == cur_trip_num:
                if cluster_id != cur_cluster:
                    cur_cluster = cluster_id
                    delta = get_times_delta(tr_times[i - 1], tr_times[i])
                    time_between_stations = 0
                    finded = 0
                    for j in range(station_num, len(stops_times)):
                        time_between_stations += int(stops_times[j])
                        if delta < time_between_stations:
                            finded = 1
                            station_num = j + 1
                            station_id = route_stops_arr[j + 1]
                            boarding_stations_nums.append(station_num)
                            boarding_stations_id.append(station_id)
                            break
                    if finded == 0:
                        station_num = len(stops_times)
                        station_id = route_stops_arr[len(stops_times)]
                        boarding_stations_nums.append(station_num)
                        boarding_stations_id.append(station_id)
                else:
                    boarding_stations_nums.append(station_num)
                    boarding_stations_id.append(station_id)
            else:
                cur_trip_num = trip_id
                cur_direction = direction_id
                curr_cluster = cluster_id
                
                if cur_direction == 0:
                    stops_times = cur_comm_df['times_for_stops1'].values[0].split(',')
                    route_stops_arr = cur_comm_df['stops_in_direction1'].values[0].split(',')
                else:
                    stops_times = cur_comm_df['times_for_stops2'].values[0].split(',')  
                    route_stops_arr = cur_comm_df['stops_in_direction2'].values[0].split(',')
                    
                station_num = 0
                station_id = route_stops_arr[0]
                boarding_stations_nums.append(station_num)
                boarding_stations_id.append(station_id)
        
                
        stops_num_column = np.array(boarding_stations_nums)
        stops_id_column = np.array(boarding_stations_id)
        
        # Этап 4 - корректировка информации о поездках
        tr_times = board_df['TRANSACT_TIME'].values
        tr_num = tr_times.shape[0]
        
        trip_nums = preliminary_trip_id_column
        directions = preliminary_direction_column
        stops_id = stops_id_column
        stops_numbers = stops_num_column
        clusters = clusters_column
        
        curr_trip = trip_nums[tr_num - 1]
        curr_dir = directions[tr_num - 1]
        curr_number = stops_numbers[tr_num - 1]
        curr_cluster = clusters[tr_num - 1]
        
        new_trips_ids = [curr_trip]
        new_directions = [curr_dir]
        new_clusters = [curr_cluster]
        new_numbers = [curr_number]
        
        pointer = tr_num - 1
        while pointer > 0:
            if (trip_nums[pointer] != trip_nums[pointer - 1]) and (stops_id[pointer] == stops_id[pointer - 1]):
                while stops_id[pointer] == stops_id[pointer - 1]:
                    new_trips_ids.append(curr_trip)
                    new_directions.append(curr_dir)
                    new_clusters.append(curr_cluster)
                    new_numbers.append(curr_number)
                    pointer -= 1
            else:
                curr_trip = trip_nums[pointer - 1]
                curr_dir = directions[pointer - 1]
                curr_cluster = clusters[pointer - 1]
                curr_number = stops_numbers[pointer - 1]
                
                new_trips_ids.append(curr_trip)
                new_directions.append(curr_dir)
                new_clusters.append(curr_cluster)
                new_numbers.append(curr_number)
                pointer -= 1
                
        # Добавление информации, полученной на 1-4 этапах в dataframe
        new_trips_column = pd.Series(np.array(list(reversed(new_trips_ids))), index=board_df.index)
        board_df.loc[:, 'TRIP_ID'] = new_trips_column
        
        new_dirs_column = pd.Series(np.array(list(reversed(new_directions))), index=board_df.index)
        board_df.loc[:, 'DIRECTION'] = new_dirs_column
        
        boarding_clusters = pd.Series(np.array(list(reversed(new_clusters))), index=board_df.index)
        board_df.loc[:, 'BOARDING_CLUSTER'] = boarding_clusters
        
        boarding_stops_nums = pd.Series(np.array(list(reversed(new_numbers))), index=board_df.index) 
        board_df.loc[:, 'BOARDING_STOP_NUM'] = boarding_stops_nums
        
        boarding_stops_ids = pd.Series(stops_id_column, index=board_df.index) 
        board_df.loc[:, 'BOARDING_STOP_ID'] = boarding_stops_ids
        
        # Сохраняем информацию
        board_df.to_csv('transactions_by_routes/' + route_num + '_route_' +  str(board_num) + '_board_itog.csv')
    
r = get_boarding_locations_by_day('92', '10.10')  

In [18]:
# Просмотр результата функции get_boarding_locations_by_day
r_df = pd.read_csv('transactions_by_routes/92_route_67_board_itog.csv')
r_df = r_df[['CARD_NUM', 'TRANSACT_DAY', 'TRANSACT_MONTH', 'TRANSACT_YEAR', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME', 'ROUTE_NUM', 'TRIP_ID', 'DIRECTION', 'BOARDING_CLUSTER', 'BOARDING_STOP_NUM', 'BOARDING_STOP_ID']]
r_df

,CARD_NUM,TRANSACT_DAY,TRANSACT_MONTH,TRANSACT_YEAR,CARRIER_BOARD_NUM,TRANSACT_TIME,ROUTE_NUM,TRIP_ID,DIRECTION,BOARDING_CLUSTER,BOARDING_STOP_NUM,BOARDING_STOP_ID
0,36116856638815400,10,10,2018,67,07:20:00,92,1,0,1,0,15952
1,36124333772784900,10,10,2018,67,07:20:00,92,1,0,1,0,15952
2,36126106787909800,10,10,2018,67,07:20:00,92,1,0,1,0,15952
3,36125887878602200,10,10,2018,67,07:23:00,92,1,0,2,1,1847
4,36116856502955200,10,10,2018,67,07:25:00,92,1,0,3,3,3485
5,36127155162926300,10,10,2018,67,07:45:00,92,2,1,1,0,22798
6,36081337792481700,10,10,2018,67,07:47:00,92,2,1,2,1,2475
7,36124338603441100,10,10,2018,67,07:47:00,92,2,1,2,1,2475
8,593641837,10,10,2018,67,07:47:00,92,2,1,2,1,2475
9,36102820687451900,10,10,2018,67,07:48:00,92,2,1,3,2,3421


In [14]:
# Вспомогательная часть кода, генерирующая полный список дат из рассматриваемого периода.
dates_array = []
aug_dates = range(1, 25)
sept_dates = range(1, 31)
oct_dates = range(1, 32)
for date in aug_dates:
    dates_array.append(str(date) + '.8')
for date in sept_dates:
    dates_array.append(str(date) + '.9')
for date in oct_dates:
    dates_array.append(str(date) + '.10')

In [15]:
# Основная функция алгоритма. Использует вспомогательные функции и для переданного маршрута рассчитывает пассажиропотоки 
# на всех транспортных средствах во все даты, в которые есть информация об этом маршруте
def extract_boarding_locations(route_num):
    for tr_date in dates_array:
        if check_date(route_num, tr_date):
            get_boarding_locations_by_day(route_num, tr_date)

In [16]:
# Применяем описанный алгоритм по очереди ко всем интересуемым маршрутам
for route_num in choosen_routes:
    extract_boarding_locations(route_num)